# Introduction

In this tutorial, we will explore how to utilize bootstrap resampling in Scikit-Learn to assess the prediction performance of Ordinary Least Squares (OLS) and Random Forest regression models to predict `balance` on the `Credit.csv` dataset. Our objective is to determine if one is (statistically) significantly better than the other.

**Outline**

1. Setup and Data Preparation: Importing libraries, loading the dataset, identifying feature types, train/test split
1. Preprocessing Pipelines: Defining pipelines for data transformation and model fitting
1. Bootstrap Resampling and Model Evaluation: Implementing bootstrap resampling to evaluate model performance
  1. Draw $B$ samples with replacement
  1. Train the two models on each sample, predict on the test-data
  1. Compare the $B$ measured RMSE values for each method.

# Load data

First we load the data file, separate the `X` from `y` column, and divide them into training and test sets.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

file_url = 'https://drive.google.com/uc?export=download&id=1YK0GV7j_2DoWSOObHRsrQ_paEhyp_iMO'
credit_data = pd.read_csv(file_url, index_col = 0)

credit_data.info()
display(credit_data.head())
display(credit_data.describe())

<class 'pandas.core.frame.DataFrame'>
Index: 400 entries, 1 to 400
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Income     400 non-null    float64
 1   Limit      400 non-null    int64  
 2   Rating     400 non-null    int64  
 3   Cards      400 non-null    int64  
 4   Age        400 non-null    int64  
 5   Education  400 non-null    int64  
 6   Gender     400 non-null    object 
 7   Student    400 non-null    object 
 8   Married    400 non-null    object 
 9   Ethnicity  400 non-null    object 
 10  Balance    400 non-null    int64  
dtypes: float64(1), int64(6), object(4)
memory usage: 37.5+ KB


,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance
1,14.891,3606,283,2,34,11,Male,No,Yes,Caucasian,333
2,106.025,6645,483,3,82,15,Female,Yes,Yes,Asian,903
3,104.593,7075,514,4,71,11,Male,No,No,Asian,580
4,148.924,9504,681,3,36,11,Female,No,No,Asian,964
5,55.882,4897,357,2,68,16,Male,No,Yes,Caucasian,331


,Income,Limit,Rating,Cards,Age,Education,Balance
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,45.218885,4735.600000,354.940000,2.957500,55.667500,13.450000,520.015000
std,35.244273,2308.198848,154.724143,1.371275,17.249807,3.125207,459.758877
min,10.354000,855.000000,93.000000,1.000000,23.000000,5.000000,0.000000
25%,21.007250,3088.000000,247.250000,2.000000,41.750000,11.000000,68.750000
50%,33.115500,4622.500000,344.000000,3.000000,56.000000,14.000000,459.500000
75%,57.470750,5872.750000,437.250000,4.000000,70.000000,16.000000,863.000000
max,186.634000,13913.000000,982.000000,9.000000,98.000000,20.000000,1999.000000


In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset

# Separate features and target variable
X = credit_data.drop('Balance', axis=1)
y = credit_data['Balance']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


*Why are we splitting before bootstrapping?*

Recall that bootstrapping *resamples with replacement*. In the process it can include copies of records. If we split into training and test after that, we can include some (copies of) data records in both training and test data. This will bias the evaluation to be too optimistic. This is an example of data leakage : leakage of test data into training invalidating the evaluation.

To avoid this, it’s better to split the data first and then apply bootstrapping only within the training set. This ensures that the model is consistently evaluated on unseen data.

# Setup the pipeline

We then create a preprocessing step to standardize numeric features and one-hot encode categorical features using a `ColumnTransformer`. Then, we create pipelines for both OLS and Random Forest models, each including the transformer and the model.

In [ ]:
# Import necessary libraries
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

# Identify numeric and categorical columns
numeric_cols = ['Age', 'Cards', 'Education', 'Income', 'Limit', 'Rating']
categorical_cols = ['Gender', 'Student', 'Married', 'Ethnicity']

# Create a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(), categorical_cols)])


In [ ]:
# Import necessary libraries
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Create pipelines for both models
ols_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])

# Fit the models
ols_pipeline.fit(X_train, y_train)
rf_pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Age', 'Cards', 'Education',
                                                   'Income', 'Limit',
                                                   'Rating']),
                                                 ('cat', OneHotEncoder(),
                                                  ['Gender', 'Student',
                                                   'Married', 'Ethnicity'])])),
                ('model', RandomForestRegressor(random_state=42))])

# Bootstrap Resampling and Model Evaluation

We draw 100 bootstrap samples (with replacement) to evaluate the Root Mean Squared Error (RMSE) of both models. In each iteration, we sample a dataset equal in size to the original data, fit our pipelines to the sampled data, evaluate them on the testing data collecting the RMSE. In the end we print the mean and standard deviation of the RMSE from both the models.

In [ ]:
# Import necessary libraries
import numpy as np
from sklearn.utils import resample
from sklearn.metrics import root_mean_squared_error

# Write a bootstrap function that trains the model on samples of training data and evaluates on test
def bootstrap(model_pipeline, X_train, y_train, X_test, y_test, n_iterations=100, seed=42):
    rmse_scores = [] # a list to hold RMSE values
    np.random.seed(seed)
    for _ in range(n_iterations):
        X_resample, y_resample = resample(X_train, y_train)
          # draw a sample of size equal to the input, with replacement: one bootstrap draw
        model_pipeline.fit(X_resample, y_resample)  # train on the resampled data
        y_pred = model_pipeline.predict(X_test)  # predict on the original test data
        rmse = root_mean_squared_error(y_test, y_pred)  # evaluate
        rmse_scores.append(rmse)  # collect for summarizing later
    return np.array(rmse_scores)  # convert list of numbers to numpy array and return

# Obtain bootstrap RMSE distributions
ols_rmse_scores = bootstrap(ols_pipeline, X_train, y_train, X_test, y_test)
rf_rmse_scores = bootstrap(rf_pipeline, X_train, y_train, X_test, y_test)

print(f'Linear Regression RMSE Mean: {ols_rmse_scores.mean():.2f}, Std Dev: {ols_rmse_scores.std():.2f}')
print(f'Random Forest RMSE Mean: {rf_rmse_scores.mean():.2f}, Std Dev: {rf_rmse_scores.std():.2f}')


Linear Regression RMSE Mean: 91.41, Std Dev: 3.62
Random Forest RMSE Mean: 135.61, Std Dev: 13.81


In this exercise the OLS has much lower RMSE than Random Forest. The standard deviations suggest that the difference in RMSEs is likely to be statistically significant.

RMSE of both models are observed at the *each sampled dataset*, leading to *pairs* of observations. A **paired t-test** is used to determine whether the mean difference between two sets of paired observations is zero.

### Paired t-Test for Bootstrapped Test Errors (Optional)



The paired t-test is a statistical test used to determine whether the mean difference between paired observations (like test errors from different bootstrapped samples) is significantly different from zero.

The difference in test errors for each bootstrap sample $i$

$$
d_i = Error_{i, Model A} - Error_{i, Model B}.
$$

represent the difference in test errors between the two models for each bootstrap sample. These differences $d_i$ are then used in the paired t-test to assess if there is a statistically significant mean difference.

Given test error differences $d_i$ for each bootstrap sample $i$, where $n$ is the number of bootstrap samples:

1. **Compute the mean of the differences**:
   $$
   \bar{d} = \frac{1}{n} \sum_{i=1}^n d_i
   $$

2. **Calculate the standard deviation of the differences**:
   $$
   s_d = \sqrt{\frac{1}{n-1} \sum_{i=1}^n (d_i - \bar{d})^2}
   $$

3. **Calculate the t-statistic**:
   $$
   t = \frac{\bar{d}}{s_d / \sqrt{n}}
   $$

The t-statistic follows a t-distribution with \( n - 1 \) degrees of freedom. We compare this value to a critical t-value or use the p-value to determine if the mean difference in test errors is statistically significant.

We can use `scipy.stats.ttest_rel` to carry out paired t-test in Python.

In [ ]:
from scipy import stats

# Statistical Testing
t_stat, p_value = stats.ttest_rel(ols_rmse_scores, rf_rmse_scores)
print(f'Paired T-test p-value: {p_value:.2e}')


Paired T-test p-value: 1.60e-54
